Demonstrate anomaly detection with mb_detect. Start with imports: 

In [2]:
%pylab

Using matplotlib backend: <object object at 0x7fddde2d66b0>
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [11]:
import pandas as pd
import torch

In [4]:
from mb_detect.models.classic import sklearn_dect
from mb_detect.models.deep.tcn_model import TCN

Get data from influxDB. This is a bit clunky, but every interface DB <-> pandas dataframe will need some custom module, so here is ous:

In [5]:
# CONFIGURATION
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

class InfluxListener:
    def __init__(self):
        token = os.environ.get("INFLUXDB_TOKEN")
        org = "manubrain"
        url = "http://influxdb:8086" # we must use the port 8086 here as we are inside the container (8000 is only from external)

        client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)
        self.client = client

    def get_data(self, window):
        # READ DATA
        query_api = self.client.query_api()

        query = """from(bucket: "sinus")
          |> range(start: -{})
          |> filter(fn: (r) => r["_measurement"] == "anomalie_detection")
          |> filter(fn: (r) => r["_field"] == "field_2" or r["_field"] == "field_0" or r["_field"] == "field_1" or r["_field"] == "field_3" or r["_field"] == "field_4" or r["_field"] == "field_5" or r["_field"] == "field_6" or r["_field"] == "field_7" or r["_field"] == "field_8" or r["_field"] == "field_9")
          |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
          """.format(window)
        ds = query_api.query_data_frame(query, org="manubrain")
        #ds.drop(["_start", "_stop", "table", "_measurement", "result"],axis=1, inplace=True)
        #ds.rename(columns = {'_time':'time'}, inplace=True)
        #ds.set_index("time", inplace=True)
        rename = {}
        [rename.update({c:c.replace("field", "value")}) for c in ds.columns if c.startswith("field")]
        ds.rename(columns=rename, inplace=True)
        return ds
listener = InfluxListener()

Use the data of the past 30 days as training data

In [6]:
# Train on the data of the last 30 days
train_data = listener.get_data("30d")
train_data

,result,table,_start,_stop,_time,_measurement,value_0,value_1,value_2,value_3,value_4,value_5,value_6,value_7,value_8,value_9
0,_result,0,2022-10-26 15:33:01.706215+00:00,2022-11-25 15:33:01.706215+00:00,2022-11-10 10:55:37.571427+00:00,anomalie_detection,0.034655,0.056094,0.192374,0.299628,0.394206,0.408738,0.603574,0.649854,0.674145,0.694070
1,_result,0,2022-10-26 15:33:01.706215+00:00,2022-11-25 15:33:01.706215+00:00,2022-11-10 10:55:38.082140+00:00,anomalie_detection,0.106831,0.135514,0.325822,0.312920,0.537146,0.478402,0.643974,0.713661,0.782807,0.743871
2,_result,0,2022-10-26 15:33:01.706215+00:00,2022-11-25 15:33:01.706215+00:00,2022-11-10 10:55:38.192851+00:00,anomalie_detection,0.229880,0.240487,0.399967,0.441518,0.501177,0.547549,0.693342,0.867549,0.810413,0.969417
3,_result,0,2022-10-26 15:33:01.706215+00:00,2022-11-25 15:33:01.706215+00:00,2022-11-10 10:55:38.298759+00:00,anomalie_detection,0.368830,0.463106,0.437331,0.662002,0.673428,0.755195,0.751918,0.906877,0.864726,1.002949
4,_result,0,2022-10-26 15:33:01.706215+00:00,2022-11-25 15:33:01.706215+00:00,2022-11-10 10:55:38.406592+00:00,anomalie_detection,0.422486,0.379451,0.640177,0.658047,0.740866,0.829094,0.897541,0.925895,1.004740,0.953258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26822,_result,0,2022-10-26 15:33:01.706215+00:00,2022-11-25 15:33:01.706215+00:00,2022-11-25 15:29:40.445327+00:00,anomalie_detection,-0.051905,0.010470,0.176002,0.244561,0.376460,0.415201,0.611532,0.736089,0.657732,0.786488
26823,_result,0,2022-10-26 15:33:01.706215+00:00,2022-11-25 15:33:01.706215+00:00,2022-11-25 15:29:41.469149+00:00,anomalie_detection,0.188229,0.263145,0.280321,0.343976,0.458292,0.505629,0.596097,0.659716,0.729402,0.802630
26824,_result,0,2022-10-26 15:33:01.706215+00:00,2022-11-25 15:33:01.706215+00:00,2022-11-25 15:29:42.493389+00:00,anomalie_detection,0.295664,0.332449,0.437544,0.555427,0.537619,0.646402,0.632945,0.685467,0.905081,0.987753
26825,_result,0,2022-10-26 15:33:01.706215+00:00,2022-11-25 15:33:01.706215+00:00,2022-11-25 15:29:43.517709+00:00,anomalie_detection,0.340843,0.433097,0.540389,0.662043,0.631107,0.769798,0.729905,0.848963,0.823540,1.024505


Train the deep model ("TCN") on the data. Epochs and with that learning time are turned down to demonstrate the workflow in short time

In [7]:
X = train_data[[c for c in train_data.columns if c.startswith("value")]]
model = TCN(X.shape[1])
model.fit(X,epochs=2) #turn up epochs for better learning

In [8]:
def make_predictions_tcn(model, timeframe):
    test_data = listener.get_data("10s")
    test_data = test_data[[c for c in train_data.columns if c.startswith("value")]]
    tx = torch.Tensor(test_data.to_numpy()).T
    history = tx[...,-6:-1]
    predict_target = tx[...,-1].numpy()
    predict_model = model.forward(history).detach().numpy().flatten()
    predict_error = np.linalg.norm(predict_target-predict_model)
    return predict_error

The model makes predictions on the newest data point based on the past 5 datapoints and we calculate the error between prediction and actual observation. A threshold (in our case 1) needs to be defined to say what is actually anominal behaviour:

In [13]:
for i in range(10):
    #test_data = make_predictions(model, "5s")
    predict_error = make_predictions_tcn(model, "5s")
    print(predict_error, predict_error > 1)
    time.sleep(1)

0.49708715 False
0.5189219 False
0.6268364 False
0.47885093 False
0.49721977 False
0.42333663 False
0.5149468 False
0.39529935 False
0.45331863 False
0.3286137 False


Direct models like Isolation Forest or SVM can also be used:

In [14]:
#model = sklearn_dect.SK_IsoForest()
model = sklearn_dect.SK_OneClassSVM()
model.fit(X)

/home/jovyan/my_repos/detect/src/mb_detect/models/classic/sklearn_dect.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._df[self._column_name] = self._labels != -1


In [15]:
def make_predictions(model, timeframe):
    test_data = listener.get_data("5s")
    pred = model.predict(test_data)
    test_data["anomaly"] = pred
    return test_data

In [16]:
for i in range(10):
    test_data = make_predictions(model, "5s")
    test_data.set_index("_time",inplace=True)
    print(test_data.anomaly)
    time.sleep(5)

_time
2022-11-25 15:41:25.760061+00:00    True
2022-11-25 15:41:26.784165+00:00    True
2022-11-25 15:41:28.350210+00:00    True
2022-11-25 15:41:29.374499+00:00    True
Name: anomaly, dtype: bool
_time
2022-11-25 15:41:30.397478+00:00     True
2022-11-25 15:41:31.422518+00:00    False
2022-11-25 15:41:32.446828+00:00     True
2022-11-25 15:41:33.471157+00:00     True
2022-11-25 15:41:34.495034+00:00    False
Name: anomaly, dtype: bool
_time
2022-11-25 15:41:35.535353+00:00    False
2022-11-25 15:41:36.559713+00:00    False
2022-11-25 15:41:37.583522+00:00    False
2022-11-25 15:41:38.607927+00:00    False
2022-11-25 15:41:39.632133+00:00    False
Name: anomaly, dtype: bool
_time
2022-11-25 15:41:40.656679+00:00    False
2022-11-25 15:41:41.679650+00:00    False
2022-11-25 15:41:42.704172+00:00    False
2022-11-25 15:41:43.728515+00:00     True
2022-11-25 15:41:44.752531+00:00     True
Name: anomaly, dtype: bool
_time
2022-11-25 15:41:45.777212+00:00    True
2022-11-25 15:41:46.801512+

KeyboardInterrupt: 